<a href="https://colab.research.google.com/github/HTTPArchive/almanac.httparchive.org/blob/main/src/tools/scripts/chart-adjustments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build

def update_chart_size(spreadsheet_id, is_dry_run = False, target_width=600, target_height=371):
    response = sheets_service.spreadsheets().get(spreadsheetId=spreadsheet_id, includeGridData=False).execute()
    sheets = response.get('sheets', [])

    for sheet in sheets:
        charts = sheet.get('charts', [])

        for chart in charts:
            chart_id = chart['chartId']
            update_request = {
                'updateEmbeddedObjectPosition': {
                    'objectId': chart_id,
                    'newPosition': {
                        'overlayPosition': {
                            'widthPixels': target_width,
                            'heightPixels': target_height
                        }
                    },
                    'fields': 'widthPixels,heightPixels'
                }
            }

            if not is_dry_run and \
                (chart['position']['overlayPosition']['widthPixels'] != target_width or chart['position']['overlayPosition']['heightPixels'] != target_height):
                sheets_service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body={'requests': [update_request]}).execute()

                print(f"""sheet: {sheet['properties']['title']},
chart: {chart['spec']['title']},
dimensions: {chart['position']['overlayPosition']['widthPixels']} x {chart['position']['overlayPosition']['heightPixels']}
                """)

    print('All the charts are sized {}px x {}px.'.format(target_width, target_height))

# Authenticate the user
auth.authenticate_user()
sheets_service = build('sheets', 'v4', cache_discovery=False)

In [ ]:
# Replace this with the ID of your Google Sheets file
SPREADSHEET_ID = '18r8cT6x9lPdM-rXvXjsqx84W7ZDdTDYGD59xr0UGOwg'

# Call the function to update the chart width
update_chart_size(SPREADSHEET_ID, is_dry_run=True)